In [26]:
# Import Libraries
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

# Load the dataset
data = pd.read_csv("/content/drive/MyDrive/data/train.csv")

# Initial data check
print("Initial data snapshot:")
print(data.head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Initial data snapshot:
   Unnamed: 0                              Name    Location  Year  \
0           1  Hyundai Creta 1.6 CRDi SX Option        Pune  2015   
1           2                      Honda Jazz V     Chennai  2011   
2           3                 Maruti Ertiga VDI     Chennai  2012   
3           4   Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013   
4           6            Nissan Micra Diesel XV      Jaipur  2013   

   Kilometers_Driven Fuel_Type Transmission Owner_Type     Mileage   Engine  \
0              41000    Diesel       Manual      First  19.67 kmpl  1582 CC   
1              46000    Petrol       Manual      First    13 km/kg  1199 CC   
2              87000    Diesel       Manual      First  20.77 kmpl  1248 CC   
3              40670    Diesel    Automatic     Second   15.2 kmpl  1968 CC   
4              86999    Diesel       M

Task (a): Handling Missing Values

In [27]:
# Check missing values in each column
missing_values = data.isnull().sum()
print("Missing Values per Column:\n", missing_values)

Missing Values per Column:
 Unnamed: 0              0
Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                  36
Seats                  38
New_Price            5032
Price                   0
dtype: int64


In [28]:
# Identify numerical and categorical columns
numerical_cols = data.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = data.select_dtypes(include=['object']).columns

print("Numerical columns:", numerical_cols)
print("Categorical columns:", categorical_cols)


Numerical columns: Index(['Unnamed: 0', 'Year', 'Kilometers_Driven', 'Seats', 'Price'], dtype='object')
Categorical columns: Index(['Name', 'Location', 'Fuel_Type', 'Transmission', 'Owner_Type',
       'Mileage', 'Engine', 'Power', 'New_Price'],
      dtype='object')


In [29]:
for col in numerical_cols:
    median_value = data[col].median()
    data[col] = data[col].fillna(median_value)
    print(f"Imputed missing values in '{col}' with median value: {median_value}")


Imputed missing values in 'Unnamed: 0' with median value: 3015.0
Imputed missing values in 'Year' with median value: 2014.0
Imputed missing values in 'Kilometers_Driven' with median value: 52576.0
Imputed missing values in 'Seats' with median value: 5.0
Imputed missing values in 'Price' with median value: 5.75


In [30]:
data.to_csv('/content/drive/MyDrive/data/results/taska.csv', index=False)

In [8]:
for col in categorical_cols:
    mode_value = data[col].mode()[0]
    data[col] = data[col].fillna(mode_value)
    print(f"Imputed missing values in '{col}' with mode value: {mode_value}")


Imputed missing values in 'Name' with mode value: Mahindra XUV500 W8 2WD
Imputed missing values in 'Location' with mode value: Mumbai
Imputed missing values in 'Fuel_Type' with mode value: Diesel
Imputed missing values in 'Transmission' with mode value: Manual
Imputed missing values in 'Owner_Type' with mode value: First
Imputed missing values in 'Mileage' with mode value: 18.9 kmpl
Imputed missing values in 'Engine' with mode value: 1197 CC
Imputed missing values in 'Power' with mode value: 74 bhp
Imputed missing values in 'New_Price' with mode value: 4.78 Lakh


In [9]:
print("\nMissing values after imputation:\n", data.isnull().sum())


Missing values after imputation:
 Unnamed: 0           0
Name                 0
Location             0
Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Mileage              0
Engine               0
Power                0
Seats                0
New_Price            0
Price                0
dtype: int64


**Why Imputation:**

Imputation preserves data: Dropping rows/columns might lead to significant data loss and reduce dataset representativeness.

Median and Mode imputation are robust methods: They maintain the original distribution and structure of the dataset, ensuring that subsequent analyses are minimally impacted.

Car datasets typically have limited data points: Dropping can significantly impact analysis quality.

Thus, imputation is a more effective strategy here.

Task (b): Remove Units from Attributes

In [10]:
print("Data before removing units:")
print(data[['Mileage', 'Engine', 'Power', 'New_Price']].head())


Data before removing units:
      Mileage   Engine      Power  New_Price
0  19.67 kmpl  1582 CC  126.2 bhp  4.78 Lakh
1    13 km/kg  1199 CC   88.7 bhp  8.61 Lakh
2  20.77 kmpl  1248 CC  88.76 bhp  4.78 Lakh
3   15.2 kmpl  1968 CC  140.8 bhp  4.78 Lakh
4  23.08 kmpl  1461 CC   63.1 bhp  4.78 Lakh


In [11]:
print("\nData Types:\n", data.dtypes)


Data Types:
 Unnamed: 0             int64
Name                  object
Location              object
Year                   int64
Kilometers_Driven      int64
Fuel_Type             object
Transmission          object
Owner_Type            object
Mileage               object
Engine                object
Power                 object
Seats                float64
New_Price             object
Price                float64
dtype: object


In [12]:
# Ensure the required columns exist
columns_to_clean = ['Mileage', 'Engine', 'Power', 'New_Price']

# Backup original columns for safety (optional)
backup = data[columns_to_clean].copy()

# Step 1: Re-clean columns with possible unit text using regex
# This is a safe step even if data looks clean; it ensures robustness
data['Mileage'] = data['Mileage'].astype(str).str.replace(' kmpl| km/kg', '', regex=True)
data['Engine'] = data['Engine'].astype(str).str.replace(' CC', '', regex=True)
data['Power'] = data['Power'].astype(str).str.replace(' bhp', '', regex=True)
data['New_Price'] = data['New_Price'].astype(str).str.replace(' Lakh', '', regex=True)

# Step 2: Convert to numeric, coercing errors to NaN
data['Mileage'] = pd.to_numeric(data['Mileage'], errors='coerce')
data['Engine'] = pd.to_numeric(data['Engine'], errors='coerce')
data['Power'] = pd.to_numeric(data['Power'], errors='coerce')
data['New_Price'] = pd.to_numeric(data['New_Price'], errors='coerce')

# Step 3: Impute missing values (if any) using median
for col in columns_to_clean:
    median_val = data[col].median()
    data[col] = data[col].fillna(median_val)
    print(f"Cleaned & imputed '{col}' with median: {median_val:.2f}")


Cleaned & imputed 'Mileage' with median: 18.19
Cleaned & imputed 'Engine' with median: 1496.00
Cleaned & imputed 'Power' with median: 98.59
Cleaned & imputed 'New_Price' with median: 4.78


In [13]:
print(data[['Mileage', 'Engine', 'Power', 'New_Price']].head())

   Mileage  Engine   Power  New_Price
0    19.67    1582  126.20       4.78
1    13.00    1199   88.70       8.61
2    20.77    1248   88.76       4.78
3    15.20    1968  140.80       4.78
4    23.08    1461   63.10       4.78


In [14]:
print("\nAfter cleaning and imputation:")
print(data[columns_to_clean].dtypes)
print("\nCheck for missing values:")
print(data[columns_to_clean].isnull().sum())



After cleaning and imputation:
Mileage      float64
Engine         int64
Power        float64
New_Price    float64
dtype: object

Check for missing values:
Mileage      0
Engine       0
Power        0
New_Price    0
dtype: int64


Task (c): One-hot Encoding for Categorical Variables

In [15]:
# Check unique values before encoding
print("Unique values in Fuel_Type:", data['Fuel_Type'].unique())
print("Unique values in Transmission:", data['Transmission'].unique())

Unique values in Fuel_Type: ['Diesel' 'Petrol' 'Electric']
Unique values in Transmission: ['Manual' 'Automatic']


In [16]:
fuel_mapping = {'Diesel': 0, 'Petrol': 1, 'Electric': 2}
data['Fuel_Type'] = data['Fuel_Type'].replace(fuel_mapping)

transmission_mapping = {'Manual': 0, 'Automatic': 1}
data['Transmission'] = data['Transmission'].replace(transmission_mapping)

data.head()

<ipython-input-16-471c45c130ac>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['Fuel_Type'] = data['Fuel_Type'].replace(fuel_mapping)
<ipython-input-16-471c45c130ac>:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['Transmission'] = data['Transmission'].replace(transmission_mapping)


,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,0,0,First,19.67,1582,126.20,5.0,4.78,12.50
1,2,Honda Jazz V,Chennai,2011,46000,1,0,First,13.00,1199,88.70,5.0,8.61,4.50
2,3,Maruti Ertiga VDI,Chennai,2012,87000,0,0,First,20.77,1248,88.76,7.0,4.78,6.00
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,0,1,Second,15.20,1968,140.80,5.0,4.78,17.74
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,0,0,First,23.08,1461,63.10,5.0,4.78,3.50


Create one more feature and add this column to the dataset (you can use mutate function in R for this). For example, you can calculate the current age of the car by subtracting “Year” value from the current year.   

In [17]:
data.head()


,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,0,0,First,19.67,1582,126.20,5.0,4.78,12.50
1,2,Honda Jazz V,Chennai,2011,46000,1,0,First,13.00,1199,88.70,5.0,8.61,4.50
2,3,Maruti Ertiga VDI,Chennai,2012,87000,0,0,First,20.77,1248,88.76,7.0,4.78,6.00
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,0,1,Second,15.20,1968,140.80,5.0,4.78,17.74
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,0,0,First,23.08,1461,63.10,5.0,4.78,3.50


In [18]:
# Import datetime to get current year
from datetime import datetime

# Get the current year
current_year = datetime.now().year
print("Current Year:", current_year)

# Create a new column 'Car_Age'
data['Car_Age'] = current_year - data['Year']

# Preview the updated dataset
print("\nData after adding 'Car_Age':")
print(data[['Year', 'Car_Age']].head())

Current Year: 2025

Data after adding 'Car_Age':
   Year  Car_Age
0  2015       10
1  2011       14
2  2012       13
3  2013       12
4  2013       12


Perform select, filter, rename, mutate, arrange and summarize with group by operations (or their equivalent operations in python) on this dataset. (4 points)  

In [19]:

required_columns = ['Name', 'Mileage', 'Engine', 'Seats']
data_selected = data[required_columns]
data_selected.head()


,Name,Mileage,Engine,Seats
0,Hyundai Creta 1.6 CRDi SX Option,19.67,1582,5.0
1,Honda Jazz V,13.00,1199,5.0
2,Maruti Ertiga VDI,20.77,1248,7.0
3,Audi A4 New 2.0 TDI Multitronic,15.20,1968,5.0
4,Nissan Micra Diesel XV,23.08,1461,5.0


In [21]:

filtered_data = data[(data['Mileage'].astype(float) > 20) & (data['Engine'].astype(float) < 1500)]
filtered_data.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Car_Age
2,3,Maruti Ertiga VDI,Chennai,2012,87000,0,0,First,20.77,1248,88.76,7.0,4.78,6.00,13
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,0,0,First,23.08,1461,63.10,5.0,4.78,3.50,12
7,9,Tata Indica Vista Quadrajet LS,Chennai,2012,65932,0,0,Second,22.30,1248,74.00,5.0,4.78,1.95,13
8,10,Maruti Ciaz Zeta,Kochi,2018,25692,1,0,First,21.56,1462,103.25,5.0,10.65,9.95,7
10,12,Maruti Swift VDI BSIV,Jaipur,2015,64424,0,0,First,25.20,1248,74.00,5.0,4.78,5.60,10


In [22]:
renamed_data = data.rename(columns={'Power_per_Liter': 'Power_per_Fuel'})
renamed_data.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Car_Age
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,0,0,First,19.67,1582,126.20,5.0,4.78,12.50,10
1,2,Honda Jazz V,Chennai,2011,46000,1,0,First,13.00,1199,88.70,5.0,8.61,4.50,14
2,3,Maruti Ertiga VDI,Chennai,2012,87000,0,0,First,20.77,1248,88.76,7.0,4.78,6.00,13
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,0,1,Second,15.20,1968,140.80,5.0,4.78,17.74,12
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,0,0,First,23.08,1461,63.10,5.0,4.78,3.50,12


In [ ]:
sorted_data = data.sort_values(by=['Age_of_Car', 'Mileage'], ascending=[True, False])
sorted_data.head()

In [23]:
data['Mileage'] = data['Mileage'].astype(float)

In [24]:
data['Avg_Mileage_Location'] = data.groupby('Location')['Mileage'].transform('mean')
data.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Car_Age,Avg_Mileage_Location
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,0,0,First,19.67,1582,126.20,5.0,4.78,12.50,10,17.886576
1,2,Honda Jazz V,Chennai,2011,46000,1,0,First,13.00,1199,88.70,5.0,8.61,4.50,14,18.238067
2,3,Maruti Ertiga VDI,Chennai,2012,87000,0,0,First,20.77,1248,88.76,7.0,4.78,6.00,13,18.238067
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,0,1,Second,15.20,1968,140.80,5.0,4.78,17.74,12,17.844200
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,0,0,First,23.08,1461,63.10,5.0,4.78,3.50,12,19.189777


In [25]:
grouped_data = data.groupby('Fuel_Type').agg({'Mileage': 'mean'}).sort_values(by='Mileage', ascending=False)
grouped_data.head()

,Mileage
Fuel_Type,
2,18.900000
0,18.652661
1,17.576509
